In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Notebook local path should be at `ScientificValueAgent/figures`.

In [ ]:
import sys
sys.path.append("..")

In [ ]:
from collections import Counter
from copy import deepcopy
from itertools import product
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pickle
from scipy.spatial import distance_matrix
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import warnings

In [ ]:
from sva import utils

Set some plotting defaults.

In [ ]:
utils.set_defaults()

In [ ]:
from sva.postprocessing import read_data, parse_results_by_acquisition_function
from sva.truth.xrd1dim import (
    _get_1d_phase_data,
    residual_1d_phase_relative_mae,
    xrd1dim_compute_metrics_all_acquisition_functions_and_LTB,
    _get_1d_phase_fractions,
    truth_xrd1dim,
    residual_1d_phase_mse
)

In [ ]:
x_grid = np.linspace(0, 100, 1000)
phases = _get_1d_phase_fractions(x_grid).T

### Average value of the GP modeling SVF

In [ ]:
from sva.experiments import Experiment, Data
from sva.value import default_asymmetric_value_function
from sva.truth.xrd1dim import truth_xrd1dim

In [ ]:
data = Data.from_initial_conditions(
    truth=truth_xrd1dim,
    value=default_asymmetric_value_function,
    seed=125,
    how="random",
    xmin=0.0,
    xmax=100.0,
    ndim=1,
    value_kwargs={"sd": None, "multiplier": 1.0},
)
experiment_UCB = Experiment(
    data=data,
    acqf_signature="botorch.acquisition:UpperConfidenceBound",
    acqf_kwargs={"beta": 0.0},
)
experiment_UCB.run(max_n_dat=50, pbar=True, record_gp_every=1, points_per_dimension_full_grid=100, fit_with_Adam=True)

In [ ]:
data = Data.from_initial_conditions(
    truth=truth_xrd1dim,
    value=default_asymmetric_value_function,
    seed=127,
    how="random",
    xmin=0.0,
    xmax=100.0,
    ndim=1,
    value_kwargs={"sd": None, "multiplier": 1.0},
)
experiment_UCB_2 = Experiment(
    data=data,
    acqf_signature="botorch.acquisition:UpperConfidenceBound",
    acqf_kwargs={"beta": 0.0},
)
experiment_UCB_2.run(max_n_dat=50, pbar=True, record_gp_every=1, points_per_dimension_full_grid=100, fit_with_Adam=True)

In [ ]:
data = Data.from_initial_conditions(
    truth=truth_xrd1dim,
    value=default_asymmetric_value_function,
    seed=131,
    how="random",
    xmin=0.0,
    xmax=100.0,
    ndim=1,
    value_kwargs={"sd": None, "multiplier": 1.0},
)
experiment_UCB_3 = Experiment(
    data=data,
    acqf_signature="botorch.acquisition:UpperConfidenceBound",
    acqf_kwargs={"beta": 0.0},
)
experiment_UCB_3.run(max_n_dat=50, pbar=True, record_gp_every=1, points_per_dimension_full_grid=100, fit_with_Adam=True)

Plot everything

In [ ]:
extent = (-0.1, 100.1, 47, 3)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(4, 3), gridspec_kw={'height_ratios':[1, 1, 1]}, sharex=True)

ax = axs[0]
ax.plot(x_grid, phases[:, 0], label="1")
ax.plot(x_grid, phases[:, 1], label="2")
ax.plot(x_grid, phases[:, 2], label="3")
ax.plot(x_grid, phases[:, 3], label="4")
ax.set_ylabel(r"$p(x)$")
# axlims = ax.get_ylim()
# ax.text(1.05, 1.0, "Phase", ha="left", va="bottom", transform=ax.transAxes)
# ax.text(0.025, 0.9, "(a)", ha="left", va="top", transform=ax.transAxes)
# ax.legend(frameon=False, bbox_to_anchor=(1.0, 0.5), loc="center left")

ax = axs[1]
svf = np.array([v["mu"] for v in experiment_UCB._record])
svf = svf[::-1, :]
svf = svf / svf.max(axis=1, keepdims=True)

ax.imshow(svf, cmap="viridis", rasterized=True, aspect="auto", extent=extent)
ytick_labels = np.array([3, 47])
ax.set_yticks(ytick_labels)
ax.set_yticklabels(ytick_labels[::-1])
ax.set_xticks(np.array([0, 20, 40, 60, 80, 100]))
ax.text(0.05, 0.9, "UCB(0)", ha="left", va="top", transform=ax.transAxes, color="white")
ax.text(0.05, 0.7, "Exp 1", ha="left", va="top", transform=ax.transAxes, color="white")
ax.set_ylabel(r"$N$")

ax = axs[2]
ax.hist(experiment_UCB.data.X.squeeze(), label="Exp 1")
ax.hist(experiment_UCB_2.data.X.squeeze(), label="Exp 2")
ax.hist(experiment_UCB_3.data.X.squeeze(), label="Exp 3")
ax.legend(fontsize=8, frameon=False)
ax.set_ylabel(r"Counts")
ax.set_xlabel(r"$x$~[a.u.]")

plt.show()
# plt.savefig("figures_xrd1dim/SI_xrd1dim_SVF_beta0.pdf", dpi=300, bbox_inches="tight")